In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
from connection import *
import datetime
import sys
import json

DB user name: maxpoon
DB address: localhost
DB port number: 5432


In [2]:
features_json_data = open('./features.json').read()
features = json.loads(features_json_data)

In [3]:
from data_extraction import *

In [4]:
stays = getICUStayPatients()

In [ ]:
stays.head(10)

In [ ]:
stays['intime'][0]

In [ ]:
getPatientTimeSeries(stays.iloc[0])

In [ ]:
getPatientTimeSeries(stays.iloc[11575])

In [ ]:
allTimeSeries = getAllPatientsTimeSeries(stays)

In [ ]:
allTimeSeries[1000].head()

In [ ]:
allTimeSeriesConcat = pd.concat(allTimeSeries)

In [ ]:
feature = 'Heart Rate'
minValue = allTimeSeriesConcat[feature].min()
maxValue = allTimeSeriesConcat[feature].max()
diff = maxValue - minValue
allTimeSeriesConcat[feature] = (allTimeSeriesConcat[feature]-minValue)/diff

In [ ]:
allTimeSeriesConcat['Heart Rate']

In [ ]:
allTimeSeries[0].columns

In [5]:
features = ['age', 'gender', 'CVP', 'Heart Rate',
       'Respiratory Rate', 'SpO2/SaO2', 'PaO2', 'paCO2', 'pH', 'Potassium',
       'Calcium', 'Glucose', 'Sodium', 'Chloride', 'HCO3', 'White Blood Cells',
       'Hemoglobin', 'Red Blood Cells', 'Platelet Count', 'Lactic Acid', 'ALT',
       'AST', 'Urea Nitrogen', 'Creatinine', 'Amylase', 'Lipase', 'Weight', 'gcs', 'ventilation', 'vasoactive medications']

In [6]:
columns = features + [feature + ' not null' for feature in features] + ['AKI']

In [7]:
allTimeSeries = fillNaForAllPatients(None)

In [8]:
allTimeSeries = [timeSeries[columns] for timeSeries in allTimeSeries]

In [ ]:
allTimeSeries[20].head(10)

In [ ]:
allTimeSeriesConcat = pd.concat(allTimeSeries)
ratio = len(allTimeSeriesConcat)/allTimeSeriesConcat['AKI'].sum()
ratio

In [9]:
from sklearn.model_selection import train_test_split

In [10]:
train, test = train_test_split(allTimeSeries, test_size=0.2, random_state=42)

In [11]:
from data_iterator import *

In [ ]:
iterator = DataIterator(allTimeSeries)

In [ ]:
x, y, sequenceLengths, musks = iterator.next_batch(64)

In [ ]:
from bidirectional_rnn import *

In [ ]:
graph = buildGraph()

In [12]:
import tensorflow as tf

/usr/local/Cellar/python3/3.6.4_1/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [ ]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [ ]:
trainGraph(graph, sess, train, test)

In [ ]:
feed = {graph['x']: x, graph['y']: y, graph['seqlen']: sequenceLengths, graph['musk']: musks}

In [ ]:
sess.run(graph['predicts'], feed_dict=feed)

In [13]:
import multi_layer_bidirectional_rnn

In [18]:
tf.reset_default_graph()

In [19]:
multi_graph = multi_layer_bidirectional_rnn.buildGraph()

In [20]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [17]:
multi_layer_bidirectional_rnn.trainGraph(multi_graph, sess, train, test)

Accuracy after epoch 1  - tr: 0.589465453572 - te: 0.616675747911
Accuracy after epoch 2  - tr: 0.613480393045 - te: 0.50823080599
Accuracy after epoch 3  - tr: 0.507416727202 - te: 0.51185574476
Accuracy after epoch 4  - tr: 0.503092188169 - te: 0.512413947947


KeyboardInterrupt: 

In [21]:
multi_layer_bidirectional_rnn.trainGraph(multi_graph, sess, train, test, 60)

Accuracy after epoch 1  - tr: 0.581913041633 - te: 0.581784937413
Accuracy after epoch 2  - tr: 0.588911185008 - te: 0.558761776722
Accuracy after epoch 3  - tr: 0.557670273905 - te: 0.566974173998
Accuracy after epoch 4  - tr: 0.573963243385 - te: 0.573427474465
Accuracy after epoch 5  - tr: 0.581991827595 - te: 0.577309206875
Accuracy after epoch 6  - tr: 0.582106895602 - te: 0.522023216304
Accuracy after epoch 7  - tr: 0.591276246347 - te: 0.59250022884
Accuracy after epoch 8  - tr: 0.594171371129 - te: 0.592560911674
Accuracy after epoch 9  - tr: 0.596643116774 - te: 0.596016729238
Accuracy after epoch 10  - tr: 0.598462322519 - te: 0.592625238481
Accuracy after epoch 11  - tr: 0.598528029491 - te: 0.592515674363
Accuracy after epoch 12  - tr: 0.59582671572 - te: 0.604460919936
Accuracy after epoch 13  - tr: 0.603183333568 - te: 0.60267880973
Accuracy after epoch 14  - tr: 0.603725405643 - te: 0.598664621094
Accuracy after epoch 15  - tr: 0.601194096275 - te: 0.597969609126
Accurac